In [8]:
import requests
import datetime, time
import lxml

In [9]:
from bs4 import BeautifulSoup

In [10]:
import pandas as pd

In [11]:
seqs = pd.read_csv("../data/sequential_id.csv")

FileNotFoundError: File b'../data/sequential_id.csv' does not exist

In [ ]:
seqs['name'] = seqs.name.str.upper()
seqs['name'] = seqs.name.replace('AURÉLIO NOMURA', 'AURELIO NOMURA')
seqs['name'] = seqs.name.replace('CAIO MIRANDA CARNEIRO', 'CAIO MIRANDA')
seqs['name'] = seqs.name.replace('ANTONIO DONATO', 'DONATO')
seqs['name'] = seqs.name.replace('EDUARDO SUPLICY', 'EDUARDO MATARAZZO SUPLICY') 
seqs['name'] = seqs.name.replace('MARIO COVAS NETO', 'MÁRIO COVAS NETO')
seqs['name'] = seqs.name.replace('REGINALDO TRIPOLI', 'REGINALDO TRÍPOLI')
seqs['name'] = seqs.name.replace('ZÉ TURIN', 'ZÉ TURIM')

In [ ]:
secretaries = pd.read_csv('../data/secretary-councilman.csv')
secretaries['name'] = seqs.name.str.upper()

In [ ]:
seqs = seqs.append(secretaries)

In [6]:
xmls = [] 
xmls.append('http://www2.camara.sp.gov.br/sisgv/Arquivos/201701.XML')
xmls.append('http://www2.camara.sp.gov.br/sisgv/Arquivos/201702.XML')

In [ ]:
lista_gastos=[]
for link in xmls:
    r = requests.get(link)
    xml = BeautifulSoup(r.content, 'xml')
    gastos = xml.find_all('TabelaPortalITEMREEMBOLSO')
    for s in gastos:
        chave = s.Chave.text
        centro_custos = s.CENTROCUSTOSID.text
        departamento = s.DEPARTAMENTO.text
        tipodep = s.TIPODEDEPARTAMENTO
        vereador = s.VEREADOR.text
        ano = s.ANO.text
        mes = s.MES.text
        despesa = s.DESPESA.text
        cnpj = s.CNPJ.text
        fornecedor = s.FORNECEDOR.text
        valor = s.VALOR.text
        data = datetime.date(int(ano), int(mes), 1)

        dict_gastos = {'DATA': data, 'Chave':chave, 'CENTROCUSTOSID': centro_custos, 'DEPARTAMENTO': departamento, 
                'TIPO_DEPARTAMENTO':tipodep, 'VEREADOR': vereador, 'DESPESA':despesa, 'CNPJ': cnpj, 
                       'FORNECEDOR': fornecedor, 'VALOR': valor}
        lista_gastos.append(dict_gastos)

In [ ]:
df = pd.DataFrame().from_records(lista_gastos)

In [ ]:
df = df[['DATA', 'VEREADOR', 'DEPARTAMENTO', 'DESPESA', 'VALOR', 'FORNECEDOR','CNPJ',]]
df[['DATA']] = df[['DATA']].apply(pd.to_datetime)
df[['VALOR']] = df[['VALOR']].apply(pd.to_numeric)
df = df.drop_duplicates()

In [ ]:
df = df.rename(columns={
    'DATA': 'date', 
    'VEREADOR': 'candidate',
    'DEPARTAMENTO': 'department', 
    'DESPESA': 'expense',
    'VALOR': 'value', 
    'FORNECEDOR': 'provider', 
    'CNPJ': 'cnpj',
})

In [ ]:
today = datetime.date.today()
df.to_csv(f"../data/{today}-expenses.csv", index=False)
